In [2]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

from langchain_core.tools import tool
import pandas as pd
from langgraph.prebuilt import create_react_agent
from pydantic.v1 import BaseModel, Field

from IPython.display import Markdown

In [3]:
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [4]:
config = {
    "master_data" : r"C:\Users\nigam\OneDrive\Documents\university_classes\AutoMMM\data\manual_data.xlsx",
    "sheet_name" : "master_data"
}

# Utility Functions

In [5]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message,tuple):
            print(message)
        else:
            message.pretty_print()
            

In [6]:
def process_config(config: dict):
    master_data = pd.read_excel(config['master_data'],sheet_name=config['sheet_name'])

    return {
        'master_data' : master_data
    }

In [8]:
# configuration = process_config(config)

# configuration['master_data'].head(3)

In [9]:
llm = ChatGroq(
    model_name="mistral-saba-24b", # llama3-70b-8192, llama-3.3-70b-versatile
    temperature=0.3
)

In [10]:
llm.invoke("hi!")

AIMessage(content="Hello! How can I assist you today? If you're up for it, let's play a game of 20 questions. You think of something, and I'll try to guess it by asking up to 20 yes-or-no questions. Sound good?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 6, 'total_tokens': 61, 'completion_time': 0.166666667, 'prompt_time': 0.001975514, 'queue_time': 0.088087122, 'total_time': 0.168642181}, 'model_name': 'mistral-saba-24b', 'system_fingerprint': 'fp_07e680a590', 'finish_reason': 'stop', 'logprobs': None}, id='run--6a351d68-0736-4e67-aeef-e052d3b14981-0', usage_metadata={'input_tokens': 6, 'output_tokens': 55, 'total_tokens': 61})

In [11]:
import os

from langchain_core.tools import tool
import pandas as pd
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from typing import Annotated, List,  Dict, Optional, Union
import operator
from pathlib import Path

from IPython.display import Markdown, Image
from langchain_experimental.utilities import PythonREPL

from langgraph.graph import START, END, StateGraph
import subprocess


llm_infograph = llm

class DataPoint(BaseModel):
    type: str 
    description: str
    value: Union[str, int, float, List[Union[str, int, float]], dict]
    context_text: Optional[str] = None

class Section(BaseModel):
    heading: str
    intro_text: str
    data_points: List[DataPoint]

class InfographicData(BaseModel):
    infographic_title: str
    sections: List[Section]


content_analyzer_llm = llm_infograph.with_structured_output(InfographicData)

print("--- ContentAnalyzerAgent: Analyzing text with LLM... ---")
raw_text = """
# LLM Response

## Market Mix Modeling Data Analysis Report

This report analyzes the provided market mix data to understand the factors influencing sales for various products.

### 1. Unique Products

The dataset includes sales data for at least three unique products: `sku_a`, `sku_b`, and `sku_c`.

### 2. Data Shape

The data is structured as a time series, with each row representing a specific date and product combination.

### 3. Column Breakdown

The data comprises 19 columns, which can be categorized as follows:

#### 3.1. Base Data

* **date:** Date of the sales record (YYYY-MM-DD format).
* **sku:** Unique identifier for each product.
* **sales:** Total sales revenue for the product on that date.
* **units:** Number of units sold for the product on that date.
* **price:** Average selling price per unit for the product on that date.

#### 3.2. Incremental Marketing Spend

* **online_spends:** Marketing spend allocated to online channels for the product on that date.
* **offline_spends:** Marketing spend allocated to offline channels for the product on that date.

#### 3.3. External Features

* **competitor_spends:** Estimated marketing spend by competitors for the product category on that date.
* **seasonality_index:** A measure of seasonal demand for the product category on that date (e.g., higher during holidays).
* **economic_indicator:** A relevant economic indicator (e.g., GDP growth, unemployment rate) that may influence sales.

#### 3.4. Social Media Marketing

* **insta_clicks:** Number of clicks on Instagram ads for the product on that date.
* **insta_spends:** Marketing spend allocated to Instagram ads for the product on that date.
* **fb_clicks:** Number of clicks on Facebook ads for the product on that date.
* **fb_spends:** Marketing spend allocated to Facebook ads for the product on that date.

#### 3.5. Other Features

* **brand_level_nonbranded_spends:** Marketing spend allocated to non-branded campaigns for the brand on that date.
* **promotions:** A binary variable indicating whether a promotion was active for the product on that date.

### 4. Business Implications

This data allows us to:

* **Measure the effectiveness of different marketing channels:** By analyzing the relationship between marketing spend (online, offline, social media) and sales, we can identify which channels are most effective for driving sales.
* **Understand the impact of external factors:** Analyzing the relationship between external factors (competitor spend, seasonality, economic indicators) and sales can help us anticipate market trends and adjust our strategies accordingly.
* **Optimize marketing budget allocation:** By understanding the return on investment (ROI) for different marketing activities, we can allocate our budget more effectively to maximize sales.
* **Identify opportunities for growth:** Analyzing the data can reveal untapped market segments or opportunities for product development.



This report provides a high-level overview of the data. Further analysis and modeling are required to draw more specific conclusions and actionable insights.

---

## Market Mix Modelling Report: SKU "sku_a"

This report analyzes the sales data for SKU "sku_a" from 2025-06-07 to 2027-05-29, focusing on identifying patterns, outliers, missing data, and correlations between key performance indicators (KPIs).

### 1. Sales Pattern Throughout the Timeline

* **General Trend:** Sales for SKU "sku_a" exhibit a fluctuating pattern over the analyzed period. 
* **Seasonality:** While a clear seasonal trend isn't immediately apparent, there are noticeable peaks and valleys in sales throughout the two-year period. Further analysis, potentially incorporating time-based features, could reveal underlying seasonal patterns.

### 2. Date Distribution and Spread

* **Even Distribution:** The data appears to be relatively evenly distributed across the available dates, with sales recorded for most days within the timeframe.

### 3. Outliers

* **Identification:**  There are no significant outliers in the sales data for SKU "sku_a" based on a visual inspection. 
* **Possible Reasons:**  The absence of outliers could indicate consistent marketing efforts and stable market conditions.
* **Impact on Sales:**  The lack of outliers suggests that the sales data is relatively stable and reliable.
* **Suggestion:**  Continue monitoring sales data for any unexpected spikes or drops that might indicate emerging outliers.

### 4. Missing Data

* **Percentage:** There are no significant instances of missing data in the provided dataset.
* **Pattern:** The data appears to be complete, with no noticeable patches or irregular gaps.
* **Possible Reasons:**  The absence of missing data suggests robust data collection practices.
* **Impact on Sales:**  The complete dataset allows for accurate analysis and modeling of sales trends.
* **Suggestion:**  Maintain consistent data collection procedures to ensure data integrity.

### 5. Count of Zeros

* **Occurrence:**  There are a few instances of zero sales recorded for SKU "sku_a".
* **Pattern:** These zero sales occurrences appear sporadic and not clustered in specific periods.
* **Percentage:** The percentage of zero sales is relatively low, indicating that sales activity is generally present.
* **Suggestion:** Investigate the reasons behind these zero sales instances. It could be due to temporary stock unavailability, promotional periods, or other market factors.

### 6. Distinct Values (Categorical Columns)

*  Please provide the categorical columns in the dataset for a detailed analysis of distinct values.

### 7. Impact of Each Column on Sales

*  A comprehensive analysis of the impact of each column on sales requires statistical modeling techniques such as regression analysis. This will allow us to quantify the contribution of each KPI to sales performance.

### 8. Inter-KPI Correlations

*  Correlation analysis will be conducted to identify relationships between different KPIs. This will help understand how marketing spend, social media activity, and other factors influence sales.

### 9. Heatmap for Correlation

*  A heatmap will be generated to visualize the correlation matrix of all KPIs. This will provide a clear and intuitive representation of the relationships between the variables.



**Next Steps:**

* Provide the categorical columns for analysis.
* Conduct regression analysis to quantify the impact of each KPI on sales.
* Generate a correlation matrix and heatmap to visualize inter-KPI relationships.



This report provides a preliminary overview of the sales data for SKU "sku_a". Further analysis and modeling will provide deeper insights into the factors driving sales performance and inform strategic decision-making.

---

## Market Mix Modelling Report: Product - sku_c

This report analyzes the provided data for product sku_c, focusing on sales patterns, data quality, and key performance indicator (KPI) relationships.

### 1. Sales Pattern Throughout the Timeline:

The sales data for sku_c shows a general upward trend over the observed period.  

* **Peak Sales:** Sales appear to peak around mid-June and mid-May of the respective years.
* **Seasonal Fluctuations:** There are noticeable seasonal fluctuations, with sales potentially higher in the summer months.

### 2. Date Distribution and Spread

The data spans from June 2025 to May 2027, with a relatively even distribution of data points across the timeframe.

### 3. Outliers

* **No significant outliers** were identified in the sales data.

### 4. Missing Data

* **Percentage of Missing Points:** The data appears to have minimal missing points. A precise percentage will require a thorough scan of the dataset.
* **Pattern of Missing Points:** No clear pattern of missing data was observed. 
* **Possible Reasons for Missing Data:** Potential reasons for any missing data could include:
    * Data entry errors
    * System glitches
    |
    * Temporary unavailability of data sources
* **Impact on Sales Analysis:** A small amount of missing data is unlikely to significantly impact the overall sales analysis.
* **Suggestion to Treat Missing Data:** If missing data points are identified, imputation techniques (e.g., mean/median imputation) could be used to fill them in.

### 5. Count of Zeros in the Data

* **Number of Zeros:** The number of zero sales values will need to be counted and analyzed.
* **Pattern of Zeros:** It's important to determine if zero sales occur in clusters or are randomly distributed.

### 6. Distinct Values (Categorical Columns)

* **List distinct values** for each categorical column (e.g., brand, product category) and their frequencies.

### 7. Impact of Each Column on Sales

* **Regression Analysis:** Conduct a regression analysis to quantify the impact of each KPI on sales. This will reveal which factors have the strongest influence on sales performance.

### 8. Inter-KPI Correlations

* **Correlation Matrix:** Calculate the correlation matrix for all KPIs. This will highlight relationships between different variables.

### 9. Heatmap for Correlation

* **Visualize Correlations:** Create a heatmap to visually represent the correlation matrix. This will make it easier to identify strong positive and negative correlations.



**Next Steps:**

* Complete the analysis of missing data, zero sales, and distinct values.
* Conduct regression analysis and generate a correlation matrix.
* Create a heatmap to visualize the correlation matrix.
* Based on the findings, develop actionable insights and recommendations for optimizing marketing spend and driving sales growth for sku_c.


"""

# Prompt for ContentAnalyzerAgent
content_analyzer_prompt = f"""
You are an expert Data Analyst and Content Summarizer. Your task is to meticulously read the provided text and extract all relevant information, structuring it into a precise JSON format for subsequent visualization. Do not interpret or design any visuals; focus solely on accurate content analysis and structuring.

**Input Text:**
{raw_text}

**Output Format (JSON Schema):**
```json
{{
    "infographic_title": "string (concise, catchy title based on main topic, max 8 words)",
    "sections": [
    {{
        "heading": "string (main heading for this section, max 10 words)",
        "intro_text": "string (brief introductory paragraph for this section, max 100 words)",
        "data_points": [
        {{
            "type": "string (e.g., 'KPI', 'Trend', 'Comparison', 'ProcessStep', 'Definition', 'TimelineEvent', 'StatisticalSummary', 'KeyInsight')",
            "description": "string (brief explanation of what this data point represents, max 20 words)",
            "value": "any (numeric value, array of strings/numbers, object, or string depending on type. For processes, use an array of strings for steps.)",
            "context_text": "string (short snippet of original text for context, max 50 words, optional)"
        }}
        ]
    }}
    ]
}}
```
**Instructions:**
1. Read the entire text carefully to grasp the overall context.
2. Identify distinct logical sections within the text. For each section, create a 'heading' and a brief 'intro_text'.
3. Within each section, identify all potential data points suitable for visualization. Extract them accurately.
4. For each 'data_point':
    * Assign the most appropriate 'type' from the suggested list.
    * Write a concise 'description'.
    * Extract its 'value'. Ensure numeric values are actual numbers, arrays are actual arrays, etc.
    * Provide 'context_text' if a short direct quote or reference from the original text helps.
5. Ensure the entire output is a single, valid, complete JSON object. Do NOT include any explanations or conversational text outside the JSON.
"""

try:
    extracted_data = content_analyzer_llm.invoke([{"role": "user", "content": content_analyzer_prompt}])
    print("Content analysis complete.")
except Exception as e:
    print(f"Error in ContentAnalyzerAgent: {e}")

--- ContentAnalyzerAgent: Analyzing text with LLM... ---
Content analysis complete.


In [13]:
import json

output = json.loads(extracted_data.model_dump_json())

C:\Users\nigam\AppData\Local\Temp\ipykernel_21116\1979893817.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  ouptut = json.loads(extracted_data.json())


In [ ]:
profile

In [ ]:
profile = df.profile_report(
    variables={
        "descriptions": {
            "date" : "Date column",
            "sku"	: "product identification number (stock keeping unit)",
            "sales" : "amount of units sold (revenue in euros)",
            "units" : "Number of units sold",
            "price" : "average price of the product in the week",
            "oos"	: "number of days the product was out of stock in a week",
            "events" : "number of events in a week",
            "product_level_branded_clicks" : "clicks generated by product based search in a week (Branded keywords) ",
            "product_level_branded_spends" : "amount of money spent on clicks generated by product based search in the week (Branded keywords)",
            "product_level_nonbranded_clicks" : "clicks generated by product based search in the week (Non - Branded keywords) ",
            "product_level_nonbranded_spends" : "amount of money spent on clicks generated by product based search in the week (Non - Branded keywords) ",
            "brand_level_branded_clicks" : "clicks generated by brand based search in the week (Branded keywords) ",
            "brand_level_branded_spends" : "amount of money spent on clicks generated by product based search in the week (Non - Branded keywords) ",
            "brand_level_nonbranded_clicks" : "clicks generated by brand based search in the week (Branded keywords) ",
            "brand_level_nonbranded_spends" : "amount of money spent on clicks generated by product based search in the week (Non - Branded keywords) ",
            "insta_clicks" : "clicks generated on instagram advertizement in the week.",
            "insta_spends" : "amount of money spent on clicks generated on instagram advertizement in the week.",
            "fb_clicks" : "clicks generated on facebook advertizement in the week.",
            "fb_spends" : "amount of money spent on clicks generated on facebook advertizement in the week."
                }
    }
    )

profile.to_file(Path("stata_auto_report.html"))

In [ ]:
profile.to_file('report.html')

In [ ]:
json_data = profile.to_json()

In [ ]:
profile.to_file("your_report.json")

In [52]:
@tool
def load_dataframe(df):
    """Load the dataframe in the context"""
    return configuration['master_data']

In [53]:
from urllib.parse import quote
from typing import Annotated, List
from typing_extensions import TypedDict
import operator
from langgraph.constants import Send
from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

In [54]:
@tool
def QuickChartTool(chart_config):
    """Generates a chart image URL from the provided Chart.js configuration."""
    chart_config_str = str(chart_config)  # ensure input is string
    url = f"https://quickchart.io/chart?c={quote(chart_config_str)}"

In [55]:
# Time vs clicks
# spends table

In [75]:
@tool
def compute_summary_statistics(data : pd.DataFrame, columns: list) -> str:
    """Compute summary statistics for specified columns in the dataset."""
    stats = data[columns].describe().to_dict()
    return str(stats)

In [ ]:
llm_with_tool = llm.bind_tools([compute_summary_statistics])

In [58]:
class Summary(BaseModel):
    purpose : str = Field(description="Summarize key findings from the raw data review to ensure data readiness for Market Mix Modeling, for the given data")
    observations : str = Field(description="""
                               Overview of data quality, completeness, and potential issues.
                               High-level insights on correlations, trends, outliers, and anomalies.
                               Recommendations for data preparation and treatment, for the given data.""")
    

    

In [77]:
class Overview(BaseModel):
    data_sources : str = Field(description="""
                               List of data sources (e.g., sales data, media spend, macroeconomic indicators).
                               Time period covered (e.g., January 2020 - December 2024).
                               Granularity (e.g., weekly, monthly, regional).""")
    observations : str = Field(description="""
                               **Target Variable**: Description of the target KPI (e.g., sales revenue, units sold).
                               **Independent Variables**: Media spends (TV, digital, print), promotions, pricing, macroeconomic factors, etc.""")
    
    data_volume : str = Field(description="Number of observations, variables, and records per product/region.")

In [78]:
class DescriptiveStats(BaseModel):
    summary_stats : str = Field(description="""
                                Mean, median, standard deviation, min, max for key variables (e.g., sales, media spends).
                                Distribution characteristics (e.g., skewness, kurtosis).""")
    market_share_analysis : str = Field(description="""
                                Market share by product/brand over time (percentage of total sales).
                                Visual: Pie chart or stacked bar chart showing market share distribution.""")
    
    spend_distribution : str = Field(description="""
                                Breakdown of media and promotional spends by channel (e.g., TV, digital, radio) in tabular form.
                                Visual: Bar chart or histogram of spend distribution across channels.
                                Insights: Identify dominant channels and variability in spend patterns.""")

In [79]:
class State(TypedDict):
    data : pd.DataFrame
    combined_summary : str
    combined_overview : str
    combined_stats : str
    combined : str

In [80]:
def write_summary(state: State):
    summary_report =  llm_with_tool.with_structured_output(Summary).invoke(f"use appropriate tool to write a detailed summary for the data : data : {state['data']}")
    purpose = summary_report.purpose
    observations = summary_report.observations
    combined_summary = f"""
## purpose 
{purpose} 
## obeservations 
{observations}
                    """
    return {'combined_summary' : combined_summary}

In [81]:
def write_overview(state: State):
    overview_report =  llm_with_tool.with_structured_output(Overview).invoke(f"{state['data']}")
    data_sources = overview_report.data_sources
    observations = overview_report.observations
    data_volume = overview_report.data_volume
    combined_overview = f"""
## data_sources 
{data_sources} 
## observations 
{observations}
## data_volume 
{data_volume}
                    """
    return {'combined_overview' : combined_overview}

In [82]:
def write_descriptive_stats(state: State):
    stats_report =  llm_with_tool.with_structured_output(DescriptiveStats).invoke(f"{state['data']}")
    summary_stats = stats_report.summary_stats
    market_share_analysis = stats_report.market_share_analysis
    spend_distribution = stats_report.spend_distribution
    combined_stats = f"""
## summary_stats 
{summary_stats} 
## market_share_analysis 
{market_share_analysis}
## spend_distribution 
{spend_distribution}
                    """
    return {'combined_stats' : combined_stats}

In [83]:
def aggregator(state: State):
    """ Combine every thing into a single format"""
    combined_text = "Here is the combined output : \n"
    combined_text += f"combined_summary: \n {state['combined_summary']}" 
    combined_text += f"combined_overview: \n {state['combined_overview']}" 
    combined_text += f"combined_stats: \n {state['combined_stats']}" 
    return {'combined': combined_text}

In [84]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

In [ ]:
parallel_builder = StateGraph(State)

parallel_builder.add_node("write_summary",write_summary)
parallel_builder.add_node("write_overview",write_overview)
parallel_builder.add_node("write_descriptive_stats",write_descriptive_stats)
parallel_builder.add_node("aggregator",aggregator)


parallel_builder.add_edge(START, "write_summary")
parallel_builder.add_edge(START, "write_overview")
parallel_builder.add_edge(START, "write_descriptive_stats")

parallel_builder.add_edge("write_summary", "aggregator")
parallel_builder.add_edge("write_overview", "aggregator")
parallel_builder.add_edge("write_descriptive_stats", "aggregator")

parallel_builder.add_edge("aggregator",END)

parallel_workflow = parallel_builder.compile()

display(Image(parallel_workflow.get_graph().draw_mermaid_png()))

In [86]:
def print_stream(stream):
    for s in stream:
        d = s[list(s.keys())[0]]
        message = d[list(d.keys())[0]]
        print(message)
            

In [87]:
output = parallel_workflow.invoke({"data":configuration['master_data']})

In [ ]:
Markdown(output['combined_summary'])

In [ ]:
print_stream(parallel_workflow.stream({"data":configuration['master_data']}))

In [ ]:
for s in :
    print(s)

    # list(s.keys())[0]

    d = s[list(s.keys())[0]]

    print(d[list(d.keys())[0]])

    
    s[list(s.keys())[0]][list(s.keys())[0]]
    import pdb; pdb.set_trace()

In [ ]:
print_stream(parallel_workflow.stream({"data":configuration['master_data']}))
    # s[list(s.keys())[0]]
# inputs = {"messages": [("user", "explain the data")]}
# print_stream(data_analysis_agent.stream(inputs, stream_mode="values"))

In [ ]:
Markdown(output['combined'])

In [172]:
class Section(BaseModel):
    name: str = Field(description="The title of the section of the report")
    description : str = Field(description="The detailed explaination of the observed or analysed number")

class Sections(BaseModel):
    sections : list[Section] = Field(description="list of all the sections")

planner = llm.with_structured_output(Sections)

In [173]:
class State(TypedDict):
    topic: str
    sections : List[Section]
    completed_sections : Annotated[
        List, operator.add
    ]
    final_report : str

class AgentState(TypedDict):
    section : Section
    completed_sections: Annotated[
        List, operator.add
        ]


def orchestrator(state: State):
    planning = planner.invoke(
        [
            SystemMessage(content = "Plan the report into subsections based on the topic."),
            HumanMessage(content = f"Here is the topic : {state['topic']}")
        ]
    )
    return {'sections': planning.sections}

def agents(state: AgentState):
    contnt = llm.invoke(
        [
            SystemMessage(content = "Generate a report section without preamble base on the given topic"),
            HumanMessage(content = f"The name : {state['section'].name}, and the description is : {state['section'].description}")
        ]
    )
    return {"completed_sections": [contnt.content]}

def agent_handler(state: State):
    return [Send("agents",{"section": s}) for s in state['sections']]

def synthesizer(state: State):
    combined_sections = state['completed_sections']
    final_report = "\n\n --- \n\n".join(combined_sections)

    return {'final_report': final_report}

In [26]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

In [ ]:
orchestrator_flow_builder = StateGraph(State)

orchestrator_flow_builder.add_node("orchestrator",orchestrator)
orchestrator_flow_builder.add_node("agents",agents)
orchestrator_flow_builder.add_node("synthesizer",synthesizer)

orchestrator_flow_builder.add_edge(START, 'orchestrator')
orchestrator_flow_builder.add_conditional_edges(
    'orchestrator',
    agent_handler,
    ["agents"]
)
orchestrator_flow_builder.add_edge("agents","synthesizer")
orchestrator_flow_builder.add_edge("synthesizer",END)


orchestrator_workflow = orchestrator_flow_builder.compile()

display(Image(orchestrator_workflow.get_graph().draw_mermaid_png()))

In [176]:
prompt = """
You are a senior data analyst specializing in Marketing Mix Modeling (MMM).
Your expertise includes:

- Decomposing sales into base, incremental, and external drivers
- ROI analysis of marketing levers (TV, digital, price cuts, etc.)
- Trend, seasonality, and anomaly detection
- Communicating insights clearly to business and marketing stakeholders
- Visualizing insights using QuickChart (https://quickchart.io/)
- Producing accurate reports in LaTeX format, suitable for PDF compilation


## Report Format 

### Market Mix Modeling: Pre-Modeling Raw Data Review Report

#### 1. Executive Summary
- **Purpose**: Summarize key findings from the raw data review to ensure data readiness for Market Mix Modeling.
- **Key Observations**:
Overview of data quality, completeness, and potential issues.
High-level insights on correlations, trends, outliers, and anomalies.
Recommendations for data preparation and treatment.

#### 2. Data Overview
- **Data Sources**:
List of data sources (e.g., sales data, media spend, macroeconomic indicators).
Time period covered (e.g., January 2020 - December 2024).
Granularity (e.g., weekly, monthly, regional).
- **Key Variables**:
**Target Variable**: Description of the target KPI (e.g., sales revenue, units sold).
**Independent Variables**: Media spends (TV, digital, print), promotions, pricing, macroeconomic factors, etc.
- **Data Volume**:
Number of observations, variables, and records per product/region.

#### 3. Descriptive Statistics
- **Summary Statistics**:
Mean, median, standard deviation, min, max for key variables (e.g., sales, media spends).
Distribution characteristics (e.g., skewness, kurtosis).
- **Market Share Analysis**:
Market share by product/brand over time (percentage of total sales).
Visual: Pie chart or stacked bar chart showing market share distribution.
- **Spend Distribution**:
Breakdown of media and promotional spends by channel (e.g., TV, digital, radio).
Visual: Bar chart or histogram of spend distribution across channels.
Insights: Identify dominant channels and variability in spend patterns.

#### 4. Correlation Analysis
- **Inter-KPI Correlation**:
Correlation matrix of independent variables (e.g., TV spend vs. digital spend).
Visual: Heatmap of correlations to identify multicollinearity risks.
Insights: Highlight high correlations (>0.7) that may require variable reduction or transformation.
- **Correlation with Target**:
Correlation coefficients between each independent variable and the target (e.g., sales).
Visual: Bar chart ranking variables by correlation strength.
Insights: Identify key drivers and weak predictors.

#### 5. Trend Analysis
- **Time Series Trends**:
Trends for target variable and key independent variables over time.
Visual: Line charts showing trends for sales, media spends, and promotions.
Insights: Identify seasonality, cyclical patterns, or structural breaks.
- **Product-Level Trends**:
Trends in sales and market share by product.
Visual: Stacked line charts or area charts for product-level trends.

#### 6. Outlier Identification
- **Methodology**:
Statistical methods used (e.g., Z-score, IQR, or domain-specific thresholds).
Example: Z-score > 3 or < -3 for sales or spend data.
- **Findings**:
List of outliers by variable (e.g., unusually high TV spend in Q3 2023).
Visual: Box plots or scatter plots highlighting outliers.
- **Impact Assessment**:
Potential impact of outliers on model performance.
Recommendations: Winsorization, capping, or removal.

#### 7. Missing Data Analysis
- **Missing Data Summary**:
Percentage of missing values by variable and time period.
Visual: Heatmap or bar chart of missing data patterns.
- **Patterns**:
Random vs. systematic missingness (e.g., missing digital spend data for specific regions).
- **Recommendations**:
Imputation methods (e.g., mean/median imputation, time-series interpolation).
Exclusion of variables with excessive missingness (>30%).

#### 8. Anomaly Detection
- **Methodology**:
Techniques used (e.g., statistical thresholds, clustering, or machine learning-based anomaly detection).
- **Findings**:
Specific anomalies (e.g., sudden sales spike in a region unrelated to marketing activity).
Visual: Time-series plots with flagged anomalies.
- **Potential Causes**:
Data entry errors, external events, or untracked campaigns.
- **Recommendations**:
Investigate anomalies with stakeholders.
Adjust data or include dummy variables for known events.

#### 9. Data Quality Issues and Treatment Suggestions
- **Summary of Issues**:
Multicollinearity, outliers, missing data, anomalies, or inconsistent granularity.
- **Proposed Treatments**:
**Multicollinearity**: Combine correlated variables or use PCA.
**Outliers**: Winsorize, cap, or remove based on domain knowledge.
**Missing Data**: Impute using time-series methods or exclude problematic variables.
**Anomalies**: Add dummy variables for known events or remove erroneous records.
**Normalization**: Scale variables (e.g., log transformation for skewed spends).
- **Next Steps**:
Validate treatments with stakeholders.
Prepare cleaned dataset for modeling.

#### 10. Appendix
- **Data Dictionary**:
Definitions and sources for all variables.
- **Additional Visualizations**:
Detailed charts or tables (e.g., raw data samples, additional trend plots).
- **Technical Notes**:
Software/tools used (e.g., Python, R, Excel for analysis).
Code snippets for key analyses (e.g., correlation matrix, outlier detection).

## Instructions

1. Data Exploration Phase
   - Load dataset using `load_dataframe`
   - Identify the dependent variable (e.g., sales)
   - Categorize independent variables as base, incremental, or external

2. Visualization Phase (Use QuickChart)
   - Generate chart URLs using the `QuickChartTool` by passing Chart.js JSON configs.
   - Embed these charts in LaTeX using the following syntax:
     \\
     \\begin{figure}[H]
     \\centering
     \\includegraphics[width=\\linewidth]{{<chart_url>}}
     \\caption{<caption_text>}
     \\end{figure}
   - Use these charts where relevant:
     - Line chart for sales trends over time
     - Bar chart for driver contributions
     - Pie chart for driver share breakdown
     - Heatmap-like correlation matrix (mock with bar/stacked bar if needed)
     - Scatter charts for bivariate analysis (optional with annotations)
     - Bar chart of missing values by column

3. Insight Generation Phase
   - Explain all chart outputs clearly in LaTeX text
   - Identify missing values, correlations, lag/adstock effects
   - Quantify driver impact
   - Highlight anomalies (peaks, dips, or seasonal trends)
   - Recommend treatment for missing data and campaign gaps

4. Report Writing Phase
   - Use LaTeX sectioning commands (\\section, \\subsection, etc.) to structure the report
   - Embed charts as figures with captions as shown above
   - Avoid hallucination or assumption — base all insights on real data
   - Provide concise bullet points using LaTeX itemize environment

## Expected Output

\\section*{Executive Summary}
- Key sales trends
- Major drivers (TV, price, etc.)
- Seasonal effects or spikes
- Anomalies or potential data issues

\\section*{Variable Classification}
- Table or list of variables: base, incremental, external with reasoning

\\section*{Data Summary}
- Describe each column and its role
- Mention missing data:
  - Count & % per column
  - Likely causes (e.g., campaign gaps, bad logging)
  - Recommended treatment (e.g., fill with 0, interpolate, drop)

\\section*{Univariate Analysis}
- Distribution patterns for each variable
- Include bar or pie charts with QuickChart embedded as figures

\\section*{Bivariate & Correlation Analysis}
- Explain relationships between drivers and sales
- Describe strength, direction, and potential multicollinearity

\\section*{Adstock & Lag Effects}
- Describe persistence/memory effects of media
- Explain where diminishing returns or delay patterns exist

\\section*{Outlier & Peak Analysis}
- Describe any anomalies
- Hypothesize missed events, overspend, or missing data

\\section*{Business Recommendations}
- Spend allocation advice (e.g., increase digital in Q2)
- Suggestions for data quality improvement
- Notes for modeling enhancements (e.g., separate halo/cannibal effects)

---
Analysis conducted by: MMM Agent \\\\
Report generated on: {current_date}

Use the following tools as needed: {tools}

Question: {input}
Thought: {agent_scratchpad}
Action: {action}
Action Input: {action_input}
Observation: {observation}
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: {final_answer}

"""


In [182]:
prompt = f"""

context : {configuration['master_data']}

## Market Mix Modeling: Pre-Modeling Raw Data Review Report

#### 1. Executive Summary
- **Purpose**: Summarize key findings from the raw data review to ensure data readiness for Market Mix Modeling.
- **Key Observations**:
Overview of data quality, completeness, and potential issues.
High-level insights on correlations, trends, outliers, and anomalies.
Recommendations for data preparation and treatment.

#### 2. Data Overview
- **Data Sources**:
List of data sources (e.g., sales data, media spend, macroeconomic indicators).
Time period covered (e.g., January 2020 - December 2024).
Granularity (e.g., weekly, monthly, regional).
- **Key Variables**:
**Target Variable**: Description of the target KPI (e.g., sales revenue, units sold).
**Independent Variables**: Media spends (TV, digital, print), promotions, pricing, macroeconomic factors, etc.
- **Data Volume**:
Number of observations, variables, and records per product/region.

#### 3. Descriptive Statistics
- **Summary Statistics**:
Mean, median, standard deviation, min, max for key variables (e.g., sales, media spends).
Distribution characteristics (e.g., skewness, kurtosis).
- **Market Share Analysis**:
Market share by product/brand over time (percentage of total sales).
Visual: Pie chart or stacked bar chart showing market share distribution.
- **Spend Distribution**:
Breakdown of media and promotional spends by channel (e.g., TV, digital, radio).
Visual: Bar chart or histogram of spend distribution across channels.
Insights: Identify dominant channels and variability in spend patterns.

#### 4. Correlation Analysis
- **Inter-KPI Correlation**:
Correlation matrix of independent variables (e.g., TV spend vs. digital spend).
Visual: Heatmap of correlations to identify multicollinearity risks.
Insights: Highlight high correlations (>0.7) that may require variable reduction or transformation.
- **Correlation with Target**:
Correlation coefficients between each independent variable and the target (e.g., sales).
Visual: Bar chart ranking variables by correlation strength.
Insights: Identify key drivers and weak predictors.

#### 5. Trend Analysis
- **Time Series Trends**:
Trends for target variable and key independent variables over time.
Visual: Line charts showing trends for sales, media spends, and promotions.
Insights: Identify seasonality, cyclical patterns, or structural breaks.
- **Product-Level Trends**:
Trends in sales and market share by product.
Visual: Stacked line charts or area charts for product-level trends.

#### 6. Outlier Identification
- **Methodology**:
Statistical methods used (e.g., Z-score, IQR, or domain-specific thresholds).
Example: Z-score > 3 or < -3 for sales or spend data.
- **Findings**:
List of outliers by variable (e.g., unusually high TV spend in Q3 2023).
Visual: Box plots or scatter plots highlighting outliers.
- **Impact Assessment**:
Potential impact of outliers on model performance.
Recommendations: Winsorization, capping, or removal.

#### 7. Missing Data Analysis
- **Missing Data Summary**:
Percentage of missing values by variable and time period.
Visual: Heatmap or bar chart of missing data patterns.
- **Patterns**:
Random vs. systematic missingness (e.g., missing digital spend data for specific regions).
- **Recommendations**:
Imputation methods (e.g., mean/median imputation, time-series interpolation).
Exclusion of variables with excessive missingness (>30%).

#### 8. Anomaly Detection
- **Methodology**:
Techniques used (e.g., statistical thresholds, clustering, or machine learning-based anomaly detection).
- **Findings**:
Specific anomalies (e.g., sudden sales spike in a region unrelated to marketing activity).
Visual: Time-series plots with flagged anomalies.
- **Potential Causes**:
Data entry errors, external events, or untracked campaigns.
- **Recommendations**:
Investigate anomalies with stakeholders.
Adjust data or include dummy variables for known events.

#### 9. Data Quality Issues and Treatment Suggestions
- **Summary of Issues**:
Multicollinearity, outliers, missing data, anomalies, or inconsistent granularity.
- **Proposed Treatments**:
**Multicollinearity**: Combine correlated variables or use PCA.
**Outliers**: Winsorize, cap, or remove based on domain knowledge.
**Missing Data**: Impute using time-series methods or exclude problematic variables.
**Anomalies**: Add dummy variables for known events or remove erroneous records.
**Normalization**: Scale variables (e.g., log transformation for skewed spends).
- **Next Steps**:
Validate treatments with stakeholders.
Prepare cleaned dataset for modeling.

#### 10. Appendix
- **Data Dictionary**:
Definitions and sources for all variables.
- **Additional Visualizations**:
Detailed charts or tables (e.g., raw data samples, additional trend plots).
- **Technical Notes**:
Software/tools used (e.g., Python, R, Excel for analysis).
Code snippets for key analyses (e.g., correlation matrix, outlier detection)."""

In [ ]:
state = orchestrator_workflow.invoke({"topic": prompt})
state

In [31]:
from IPython.display import Markdown

# Markdown(state['final_report'])

In [28]:
data_analysis_agent = create_react_agent(
    model = llm,
    tools = [load_dataframe, QuickChartTool],
    prompt = prompt,
    name = "data_analysis_agent",
)

In [29]:
#prompt
# task
# context
# example
# persona
# format
# tone

In [ ]:
inputs = {"messages": [("user", "explain the data")]}
# print_stream(data_analysis_agent.stream(inputs, stream_mode="values"))
data_analysis_agent.invoke(inputs)

In [ ]:
no_of_weeks = 104

In [1]:
# #TODO: 
# 1. RAG load - https://www.latentview.com/wp-content/uploads/2019/08/Marketing-Mix-Model.pdf
# 2. Generate prompt
# 3. react agent to dataAnalysis
# 4. crewai


In [ ]:
from crewai_tools import RagTool

# Create a RAG tool with default settings
rag_tool = RagTool()

# Add content from a file
rag_tool.add(data_type="file", path="path/to/your/document.pdf")

# Add content from a web page
# rag_tool.add(data_type="web_page", url="https://example.com")
# https://develop.nielsen.com/wp-content/uploads/sites/2/2019/04/marketing-mix-modeling-what-marketers-need-to-know.pdf

# Define an agent with the RagTool
@agent
def knowledge_expert(self) -> Agent:
    '''
    This agent uses the RagTool to answer questions about the knowledge base.
    '''
    return Agent(
        config=self.agents_config["knowledge_expert"],
        allow_delegation=False,
        tools=[rag_tool]
    )